In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fer2013:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F28577%2F36420%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240221%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240221T131726Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3e3fc5f5e50026b4983a4ea6e46652443df96192dd7e4c64181ed31b7e099713e19815f33a10bcb5aee07be429b3921c349d72809b6bf332d543cae014e78044a8d8d98f9ac7a47b81dbcc72eaff5dc73db95d68d4560eccbbea781ea235a1c3bb999cce78bdbaeb3124961b6ef5f1965eb26b06e9bcfd39281e7661a5d266389cb938e661ccc1ff0f77447a48ce65e9a88ed8f736c54d215e116cb93477703ce37e3f5b3fd5db7431d18732adf4e0bbeb481f33e82ea93f04de8af0c8bcdb55ef176019185354845a25df27c86b105628fd46729b33586f3df170d43158cd86a0088f1a24b9e59c99149e99add23961cb8751f3155f3fee628b67044b62e721'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/fer2013/fer2013.csv')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# İlk birkaç satırı incele
df.head()

In [ ]:
unique_usages = df['Usage'].unique()
unique_usages

In [ ]:
unique_usages = df['Usage'].value_counts()

# Bar grafiği oluştur
plt.figure(figsize=(8, 6))
sns.barplot(x=unique_usages.index, y=unique_usages.values, palette="viridis")
plt.title('Usage Sütunundaki Benzersiz Değerlerin Sayısı')
plt.xlabel('Usage')
plt.ylabel('Sayı')
plt.show()

In [ ]:
# 'Usage' sütununa göre train ve test setlerini ayırma
train_set = df[df['Usage'] == 'Training']
test_set = df[df['Usage'] == 'PrivateTest']

# Train setinin bilgilerini göster
print("Train setinin bilgileri:")
print(train_set.info())

# Test setinin bilgilerini göster
print("\nTest setinin bilgileri:")
print(test_set.info())

In [ ]:
# Duyguların dağılımını görselleştir
plt.figure(figsize=(8, 6))
sns.countplot(x='emotion', data=df)
plt.title('Duyguların Dağılımı')
plt.show()

In [ ]:
# Örnek olarak 0.indeksteki görüntünün piksel değerlerini al
sample_pixels = df['pixels'][0]

# Piksel değerlerini diziye dönüştür
pixel_array = np.array(sample_pixels.split(), dtype='int')

# Görüntüyü yeniden şekillendir
image = pixel_array.reshape(48, 48)

# Görüntüyü görselleştir
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
pixels = df['pixels'].tolist()
X = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(48, 48)
    X.append(face.astype('float32') / 255.0)

In [ ]:
X = np.array(X)
X = np.expand_dims(X, -1) #boyut ekleme

In [ ]:
y = df['emotion'].values


TODO: preprocess_data fonksiyonu yaz.

In [ ]:
#bu işlemleri tek bir fonksiyonda toplamak daha sağlıklı olacak
def preprocess_data(data):
    """
    Veri ön işleme fonksiyonu:
    - 'pixels' sütunundan görüntü verilerini çıkartma
    - Görüntü verilerini normalize etme ve uygun şekle 48,48
    - Etiket verilerini 'emotion' sütunundan çıkartma

    Parameters:
    - data: İşlenecek DataFrame

    Returns:
    - X: Giriş verileri (normalize ve uygun şekilde)
    - y: Etiket verileri
    """

    # 'pixels' sütununu kullanarak görüntü verilerini çıkartma
    pixel_lists = df['pixels'].tolist()
    images = []

    # Her bir piksel dizisini işleme
    for pixel_sequence in pixel_lists:
        pixel_values = [int(pixel) for pixel in pixel_sequence.split(' ')]
        image = np.asarray(pixel_values).reshape(48, 48)
        images.append(image.astype('float32') / 255.0)

    # Giriş verilerini NumPy dizisine dönüştürme
    X = np.array(images)

    # Giriş verilerine kanal boyutu ekleyerek uygun şekle getirme
    X = np.expand_dims(X, -1)

    # Etiket verilerini 'emotion' sütunundan çıkartma
    y = df['emotion'].values

    return X, y


In [ ]:
# Eğitim ve test setlerini oluşturduktan sonra
X_train, y_train = preprocess_data(train_set)
X_test, y_test = preprocess_data(test_set)


In [ ]:
#kerası import edelim
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
#relu aktivasyon , input shape i 48 48 1. 1-->gri
model.add(MaxPooling2D((2, 2))) #Maksimum havuzlama (max pooling) katmanı.
model.add(Conv2D(64, (3, 3), activation='relu')) #ikinci katman. burada 64 adet
model.add(MaxPooling2D((2, 2)))
model.add(Flatten()) #Flatten katmanı

#Tam bağlantılı bi gizli katman ekler. Bu katmanda 64 nöron bulunur,ReLU aktivasyon fonksiyonu kullanılır.
model.add(Dense(64, activation='relu'))

#Çıkış katmanı. Bu, 7 sınıflı bir duygu problemi için tasarlama için.
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Softmax aktivasyon fonksiyonu, çıkıştaki değerleri olasılıklara dönüştürür.her bir sınıfa ait olasılıkları elde etmenizi sağlar. Örneğin, bir görüntü için çıkış katmanındaki 7 nöronun değerleri 7 adet olur. Softmax aktivasyonu bu değerleri **normalize** eder, böylece toplam olasılık 1 olur ve bu, sınıflar arasında karşılaştırma yapmayı kolaylaştırır.

bu değerlere göre girdinin hangi duyguda baskın olduğunu (oranı) tespit edeceğiz.

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

In [ ]:
#test veri setini
X_test, y_test = preprocess_data(test_set)

In [ ]:

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"\nTest Accuracy: {test_acc}")


epochs=10 iken sonuçlar:

Test Loss: 1.4082423448562622

Test Accuracy: 0.5210365056991577

epochs=20 iken sonuçlar:

Test Loss: 4.1522979736328125

Test Accuracy: 0.4998606741428375

**Sonuç olarak, bu durumda modelin performansının düştüğü ve daha önceki durumdan daha kötü tahminler yaptığı söylenebilir. **


epochs=5 iken sonuçlar:

Test Loss: 4.579214572906494

Test Accuracy: 0.4920590817928314


şuan denemelerimdeki en verimli 10 old. için 10da çalıştırıcam.

todo: grid search fonksiyonuyla en iyi hiperparametre kombinasyonları bulunabilir.

In [ ]:
# Eğitim sürecindeki metrikleri içeren geçmiş
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Kayıp ve doğruluk grafiği çizimi
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()


In [ ]:
# Modelin tahminleri
predictions = model.predict(X_test)

# İlk 20 örneği inceleme
for i in range(20):
    predicted_label = np.argmax(predictions[i])
    true_label = y_test[i]

    print(f"Örnek {i + 1} - Tahmin: {predicted_label}, Gerçek Etiket: {true_label}")


TODO: karmaşıklık matrisinde gösterim ekle